In [1]:
import biosignalsnotebooks as bsnb
import numpy as np

In [2]:
data, header = bsnb.load_signal("https://drive.google.com/open?id=1XcYTmw5ZGoKBGNq1xM4FbFnoN4XORjwW", get_header=True)

In [3]:
data

{'CH1': array([33484., 33792., 34192., ..., 28277., 28152., 27902.])}

In [4]:
header

{'sensor': ['EEG'],
 'device name': '00:07:80:79:6F:DB',
 'sync interval': 2,
 'time': '16:44:2.887',
 'comments': '',
 'device connection': 'BTH00:07:80:79:6F:DB',
 'channels': [1],
 'keywords': '',
 'convertedValues': 0,
 'digital IO': [0, 1],
 'firmware version': 772,
 'device': 'channeller',
 'sampling rate': 1000,
 'resolution': [16],
 'date': '2018-7-4',
 'column labels': {1: 2}}

## ECG Signals

In [10]:
import pickle

In [11]:
def spectralCentroid(X):
    """Computes spectral centroid of frame (given abs(FFT))"""
    L = X.shape[0]
    ind = (np.arange(1, len(X) + 1)) * (100/(2.0 * len(X)))
    Xt = X.copy()
    Xt = Xt / Xt.max()
    NUM = np.sum(ind * Xt)
    DEN = np.sum(Xt) + 0.000000001
    # Centroid:
    C = (NUM / DEN)
    return C

def stSpectralRollOff(X, c):
    """Computes spectral roll-off"""
    totalEnergy = np.sum(X ** 2)
    fftLength = len(X)
    Thres = c*totalEnergy
    # Ffind the spectral rolloff as the frequency position where the respective spectral energy is equal to c*totalEne    rgy
    CumSum = np.cumsum(X ** 2) + 0.00000001
    [a, ] = np.nonzero(CumSum > Thres)
    if len(a) > 0:
        mC = np.float64(a[0]) / (float(fftLength))
    else:
        mC = 0.0
    return (mC)

In [12]:
def fileFeatureExtraction(dat, signal_type):
    """ feature extraction from file"""
    # b = np.load(fileName)
    b= dat
    c = np.array(b[signal_type])

    rawData = c.astype("float64")
    rawData[np.isnan(rawData)] = 0.000000001
    #rawData = np.nan_to_num(rawData)


    means = rawData.mean(axis = 0)             # compute average
    stds = rawData.std(axis = 0)               # compute std
    maxs = rawData.max(axis = 0)               # compute max values
    mins = rawData.min(axis = 0)               # compute min values
    centroid = []
    rolloff = []

    # print(rawData)
    # print(fileName)
    for f in range(rawData.shape[1]):          # compute spectral features
        fTemp = abs(fft(rawData[:,f]))         # compute FFT
        fTemp = fTemp[0:int(fTemp.shape[0]/2)]  # get the first symetrical FFT part
        c = 0.9999
        if len(fTemp) == 0:
            pass
        else:
            # if np.isnan(fTemp).any():
            #     print (fTemp, fileName)
            centroid.append(spectralCentroid(fTemp))    # compute spectral centroid
            rolloff.append(stSpectralRollOff(fTemp, c)) # compute spectral rolloff

    if len(means) == 4 and len(stds) ==4 and len(maxs) == 4 and len(mins)== 4 and len(centroid) == 4 and len(rolloff) ==4:
        featureVector = np.concatenate((means, stds, maxs, mins, centroid, rolloff))  # concatenate features to form the final feature vector
        return featureVector

In [28]:
sample_eeg_path = "/home/ashish/Documents/github/VA/data/cognitive_data/user_6/session_1/block6_2/eeg/6_6_2"

In [29]:
from scipy.io import loadmat

In [30]:
file = loadmat(sample_eeg_path, mat_dtype=True)

ValueError: Unknown mat file type, version 56, 54